In [1]:
%matplotlib inline
import importlib, utils2; importlib.reload(utils2)
from utils2 import *

Using TensorFlow backend.


In [2]:
np.set_printoptions(4)
cfg= K.tf.ConfigProto(gpu_options={'allow_growth':True})
K.set_session(K.tf.Session(config=cfg))

In [3]:
def tokenize(sent):
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [4]:
def parse_stories(lines):
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        if int(nid) == 1: story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            substory = [[str(i)+":"]+x for i,x in enumerate(story) if x]
            data.append((substory, q, a))
            story.append('')
        else: story.append(tokenize(line))
    return data

In [5]:
path = get_file('babi-tasks-v1-2.tar.gz', 
                origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
tar = tarfile.open(path)

In [128]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
    'two_supporting_facts_1k': 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge_type = 'two_supporting_facts_10k'
challenge = challenges[challenge_type]

In [129]:
def get_stories(f):
    data = parse_stories(f.readlines())
    return [(story, q, answer) for story, q, answer in data]

In [130]:
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

/home/ubuntu/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [131]:
stories = train_stories + test_stories

In [132]:
len(stories)

11000

In [133]:
stories[0]

([['0:', 'Mary', 'moved', 'to', 'the', 'bathroom', '.'],
  ['1:', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.'],
  ['2:', 'Mary', 'got', 'the', 'football', 'there', '.'],
  ['3:', 'John', 'went', 'to', 'the', 'kitchen', '.'],
  ['4:', 'Mary', 'went', 'back', 'to', 'the', 'kitchen', '.'],
  ['5:', 'Mary', 'went', 'back', 'to', 'the', 'garden', '.']],
 ['Where', 'is', 'the', 'football', '?'],
 'garden')

In [134]:
story_maxlen=max((len(s) for x, _, _ in stories for s in x))
story_maxsents = max((len(x) for x, _, _ in stories))
query_maxlen = max(len(x) for _, x, _ in stories)


In [135]:
def do_flatten(el):
    return isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes))
def flatten(l):
    for el in l:
        if do_flatten(el): yield from flatten(el)
        else: yield el

In [136]:
# Create vocabulary of corpus and find size, including a padding element.
vocab = sorted(set(flatten(stories)))
vocab.insert(0, '<PAD')
print(vocab)
vocab_size=len(vocab)

['<PAD', '.', '0:', '10:', '11:', '12:', '13:', '14:', '15:', '16:', '17:', '18:', '19:', '1:', '20:', '21:', '22:', '23:', '24:', '25:', '26:', '27:', '28:', '29:', '2:', '30:', '31:', '32:', '33:', '34:', '35:', '36:', '37:', '38:', '39:', '3:', '40:', '41:', '42:', '43:', '44:', '45:', '46:', '47:', '48:', '49:', '4:', '50:', '51:', '52:', '53:', '54:', '55:', '56:', '57:', '58:', '59:', '5:', '60:', '61:', '62:', '63:', '64:', '65:', '66:', '67:', '68:', '69:', '6:', '70:', '71:', '72:', '73:', '74:', '75:', '76:', '77:', '78:', '79:', '7:', '81:', '82:', '84:', '85:', '87:', '88:', '8:', '90:', '91:', '9:', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'apple', 'back', 'bathroom', 'bedroom', 'discarded', 'down', 'dropped', 'football', 'garden', 'got', 'grabbed', 'hallway', 'is', 'journeyed', 'kitchen', 'left', 'milk', 'moved', 'office', 'picked', 'put', 'the', 'there', 'to', 'took', 'travelled', 'up', 'went']


In [137]:
vocab_size

124

In [138]:
story_maxsents, vocab_size, story_maxlen, query_maxlen, len(train_stories), len(test_stories)

(88, 124, 8, 5, 10000, 1000)

In [67]:
''' can backward through the sentences to find the answer to the question '''

' can backward through the sentences to find the answer to the question '

In [139]:
# create index mapping for the vocabulary
word_idx = dict((c, i) for i, c in enumerate(vocab))
word_idx

{'.': 1,
 '0:': 2,
 '10:': 3,
 '11:': 4,
 '12:': 5,
 '13:': 6,
 '14:': 7,
 '15:': 8,
 '16:': 9,
 '17:': 10,
 '18:': 11,
 '19:': 12,
 '1:': 13,
 '20:': 14,
 '21:': 15,
 '22:': 16,
 '23:': 17,
 '24:': 18,
 '25:': 19,
 '26:': 20,
 '27:': 21,
 '28:': 22,
 '29:': 23,
 '2:': 24,
 '30:': 25,
 '31:': 26,
 '32:': 27,
 '33:': 28,
 '34:': 29,
 '35:': 30,
 '36:': 31,
 '37:': 32,
 '38:': 33,
 '39:': 34,
 '3:': 35,
 '40:': 36,
 '41:': 37,
 '42:': 38,
 '43:': 39,
 '44:': 40,
 '45:': 41,
 '46:': 42,
 '47:': 43,
 '48:': 44,
 '49:': 45,
 '4:': 46,
 '50:': 47,
 '51:': 48,
 '52:': 49,
 '53:': 50,
 '54:': 51,
 '55:': 52,
 '56:': 53,
 '57:': 54,
 '58:': 55,
 '59:': 56,
 '5:': 57,
 '60:': 58,
 '61:': 59,
 '62:': 60,
 '63:': 61,
 '64:': 62,
 '65:': 63,
 '66:': 64,
 '67:': 65,
 '68:': 66,
 '69:': 67,
 '6:': 68,
 '70:': 69,
 '71:': 70,
 '72:': 71,
 '73:': 72,
 '74:': 73,
 '75:': 74,
 '76:': 75,
 '77:': 76,
 '78:': 77,
 '79:': 78,
 '7:': 79,
 '81:': 80,
 '82:': 81,
 '84:': 82,
 '85:': 83,
 '87:': 84,
 '88:': 85,

In [140]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []; Xq = []; Y = []
    for story, query, answer in data:
        x = [[word_idx[w] for w in s] for s in story]
        xq = [word_idx[w] for w in query]
        y = [word_idx[answer]]
        X.append(x); Xq.append(xq); Y.append(y)
    return ([pad_sequences(x, maxlen=story_maxlen) for x in X],
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [141]:
inputs_train, queries_train, answers_train = vectorize_stories(train_stories, 
     word_idx, story_maxlen, query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories, 
     word_idx, story_maxlen, query_maxlen)

In [142]:
len(inputs_train[0])
inputs_train[0]

array([[  0,   2,  93, 113, 119, 117,  98,   1],
       [  0,  13,  94, 109, 119, 117,  99,   1],
       [  0,  24,  93, 105, 117, 103, 118,   1],
       [  0,  35,  92, 123, 119, 117, 110,   1],
       [ 46,  93, 123,  97, 119, 117, 110,   1],
       [ 57,  93, 123,  97, 119, 117, 104,   1]], dtype=int32)

In [143]:
(story_maxsents-it.shape[0],story_maxlen)

(78, 8)

In [144]:
'''padding the matrix to have the same shape'''
def stack_inputs(inputs):
    for i,it in enumerate(inputs):
        inputs[i] = np.concatenate([it, 
                           np.zeros((story_maxsents-it.shape[0],story_maxlen), 'int')])
    return np.stack(inputs)
inputs_train = stack_inputs(inputs_train)
inputs_test = stack_inputs(inputs_test)

In [146]:
inps = [inputs_train, queries_train]
val_inps = [inputs_test, queries_test]

In [147]:
len(inps), len(val_inps)
queries_train.shape, queries_test.shape

((10000, 5), (1000, 5))

In [148]:
queries_train[0,:]

array([ 95, 108, 117, 103,  90], dtype=int32)

In [149]:
inputs_train.shape

(10000, 88, 8)

# Model

In [91]:
# adding up the word embeddings
emb_dim = 20

In [92]:
parms = {'verbose': 2, 'callbacks': [TQDMNotebookCallback(leave_inner=False)]}

In [94]:
# The embedding works as desired; the raw input has 10 sentences of 8 words, 
# and the output has 10 sentence embeddings of length 20.

In [95]:
'''TimeDistributed here to apply the embedding to every element of the sequence, then the Lambda layer adds them up'''
def emb_sent_bow(inp):
    emb = TimeDistributed(Embedding(vocab_size, emb_dim))(inp)
    return Lambda(lambda x: K.sum(x, 2))(emb)

In [96]:
inp_story = Input((story_maxsents, story_maxlen))
emb_story = emb_sent_bow(inp_story)
inp_story.shape, emb_story.shape

(TensorShape([Dimension(None), Dimension(10), Dimension(8)]),
 TensorShape([Dimension(None), Dimension(10), Dimension(20)]))

In [97]:
#  We do the same for the queries, omitting the TimeDistributed since there is only one query. 
# We use Reshape to match the rank of the input.

In [101]:
vocab_size, emb_dim, Input((query_maxlen,)).shape

(32, 20, TensorShape([Dimension(None), Dimension(4)]))

In [106]:
inp_q = Input((query_maxlen,))
emb_q = Embedding(vocab_size, emb_dim)(inp_q)
print(emb_q.shape)
emb_q = Lambda(lambda x: K.sum(x, 1))(emb_q)
print(emb_q.shape)
emb_q = Reshape((1, emb_dim))(emb_q)
print(emb_q.shape)
inp_q.shape, emb_q.shape

(?, 4, 20)
(?, 20)
(?, 1, 20)


(TensorShape([Dimension(None), Dimension(4)]),
 TensorShape([Dimension(None), Dimension(1), Dimension(20)]))

In [107]:
emb_story.shape, emb_q.shape

(TensorShape([Dimension(None), Dimension(10), Dimension(20)]),
 TensorShape([Dimension(None), Dimension(1), Dimension(20)]))

In [108]:
x = merge([emb_story, emb_q], mode='dot', dot_axes=2)
x = Reshape((story_maxsents,))(x)
print(x.shape)
x = Activation('softmax')(x)
print(x.shape)
match = Reshape((story_maxsents,1))(x)
match.shape

(?, 10)
(?, 10)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


TensorShape([Dimension(None), Dimension(10), Dimension(1)])

In [110]:
emb_c = emb_sent_bow(inp_story)
x = merge([match, emb_c], mode='dot', dot_axes=1)
print(x.shape)
response = Reshape((emb_dim,))(x)
print(response.shape)
res = Dense(vocab_size, activation='softmax')(response)
res.shape

(?, 1, 20)
(?, 20)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


TensorShape([Dimension(None), Dimension(32)])

In [111]:
answer = Model([inp_story, inp_q], res)

In [113]:
answer.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 4)             0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 10, 8)         0                                            
____________________________________________________________________________________________________
embedding_6 (Embedding)          (None, 4, 20)         640         input_8[0][0]                    
____________________________________________________________________________________________________
time_distributed_1 (TimeDistribu (None, 10, 8, 20)     640         input_1[0][0]                    
___________________________________________________________________________________________

In [114]:
answer.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [115]:
K.set_value(answer.optimizer.lr, 1e-2)
hist=answer.fit(inps, answers_train, **parms, nb_epoch=4, batch_size=32,
                validation_data=(val_inps, answers_test))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 10000 samples, validate on 1000 samples


Epoch 1/4
1s - loss: 0.4366 - acc: 0.8475 - val_loss: 6.9708e-05 - val_acc: 1.0000


Epoch 2/4
1s - loss: 0.0076 - acc: 0.9983 - val_loss: 2.0632e-04 - val_acc: 1.0000


Epoch 3/4
1s - loss: 0.0066 - acc: 0.9989 - val_loss: 0.1009 - val_acc: 0.9850


Epoch 4/4
1s - loss: 0.0060 - acc: 0.9989 - val_loss: 5.6249e-06 - val_acc: 1.0000



In [116]:
f = Model([inp_story, inp_q], match)

In [117]:
qnum=6

In [118]:
l_st = len(train_stories[qnum][0])+1
train_stories[qnum]

([['0:', 'Sandra', 'travelled', 'to', 'the', 'office', '.'],
  ['1:', 'Sandra', 'went', 'to', 'the', 'bathroom', '.'],
  ['3:', 'Mary', 'went', 'to', 'the', 'bedroom', '.'],
  ['4:', 'Daniel', 'moved', 'to', 'the', 'hallway', '.']],
 ['Where', 'is', 'Sandra', '?'],
 'bathroom')

In [121]:
np.squeeze(f.predict([inputs_train[qnum:qnum+1], queries_train[qnum:qnum+1]]))[:l_st]

array([  2.4209e-02,   9.7572e-01,   1.2684e-05,   6.2878e-05,   1.1691e-09], dtype=float32)

In [122]:
answers_train[qnum:qnum+10,0]


array([19, 19, 27, 22, 19, 20, 19, 19, 20, 20])

In [123]:
np.argmax(answer.predict([inputs_train[qnum:qnum+10], queries_train[qnum:qnum+10]]), 1)

array([19, 19, 27, 22, 19, 20, 19, 19, 20, 20])

In [124]:
answer.predict([inputs_train[qnum:qnum+1], queries_train[qnum:qnum+1]])

array([[  1.2020e-12,   7.6779e-13,   1.3014e-12,   8.0154e-13,
          7.8482e-13,   5.9923e-13,   7.9556e-13,   8.9453e-13,
          2.5134e-12,   1.2741e-12,   1.2244e-12,   3.9635e-13,
          7.2166e-13,   4.6996e-13,   8.0795e-13,   1.0314e-12,
          7.7528e-13,   5.7153e-13,   6.4650e-13,   1.0000e+00,
          8.8141e-11,   4.6416e-11,   4.0828e-12,   1.0003e-12,
          6.0218e-13,   1.4031e-10,   1.3553e-12,   4.7173e-11,
          8.7698e-13,   7.1562e-13,   4.2013e-13,   6.9185e-13]], dtype=float32)

In [125]:

vocab[19]

'bathroom'

# TWO surpport

In [150]:
inputs_train.shape, inputs_test.shape

((10000, 88, 8), (1000, 88, 8))

In [151]:

parms = {'verbose': 2, 'callbacks': [TQDMNotebookCallback(leave_inner=False)]}
emb_dim = 30

In [152]:
def emb_sent_bow(inp):
    emb_op = TimeDistributed(Embedding(vocab_size, emb_dim))
    emb = emb_op(inp)
    emb = Lambda(lambda x: K.sum(x, 2))(emb)
#     return Elemwise(0, False)(emb), emb_op
    return emb, emb_op

In [153]:
inp_story = Input((story_maxsents, story_maxlen))
inp_q = Input((query_maxlen,))

In [154]:
emb_story, emb_story_op = emb_sent_bow(inp_story)

In [155]:
emb_q = emb_story_op.layer(inp_q)
emb_q = Lambda(lambda x: K.sum(x, 1))(emb_q)


In [156]:
h = Dense(emb_dim)


In [159]:
def one_hop(u, A):
    C, _ = emb_sent_bow(inp_story)
    x = Reshape((1, emb_dim))(u)
    x = merge([A, x], mode='dot', dot_axes=2)
    x = Reshape((story_maxsents,))(x)
    x = Activation('softmax')(x)
    match = Reshape((story_maxsents,1))(x)

    x = merge([match, C], mode='dot', dot_axes=1)
    x = Reshape((emb_dim,))(x)
    x = h(x)
    x = merge([x, emb_q], 'sum')
    return x, C

In [160]:
response, emb_story = one_hop(emb_q, emb_story)
response, emb_story = one_hop(response, emb_story)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  after removing the cwd from sys.path.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e

In [161]:
res = Dense(vocab_size, activation='softmax')(response)

In [162]:
answer = Model([inp_story, inp_q], res)
answer.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [164]:
answer.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_10 (InputLayer)            (None, 5)             0                                            
____________________________________________________________________________________________________
input_9 (InputLayer)             (None, 88, 8)         0                                            
____________________________________________________________________________________________________
embedding_9 (Embedding)          (None, 5, 30)         3720        input_10[0][0]                   
____________________________________________________________________________________________________
time_distributed_4 (TimeDistribu (None, 88, 8, 30)     3720        input_9[0][0]                    
___________________________________________________________________________________________

In [165]:
K.set_value(answer.optimizer.lr, 5e-3)
hist=answer.fit(inps, answers_train, **parms, nb_epoch=8, batch_size=32,
           validation_data=(val_inps, answers_test))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 10000 samples, validate on 1000 samples


Epoch 1/8
2s - loss: 1.6987 - acc: 0.3014 - val_loss: 1.3912 - val_acc: 0.4800


Epoch 2/8
2s - loss: 1.0302 - acc: 0.6133 - val_loss: 0.8364 - val_acc: 0.6890


Epoch 3/8
2s - loss: 0.7647 - acc: 0.7253 - val_loss: 0.7557 - val_acc: 0.7500


Epoch 4/8
2s - loss: 0.6425 - acc: 0.7721 - val_loss: 0.7376 - val_acc: 0.7170


Epoch 5/8
2s - loss: 0.5543 - acc: 0.8057 - val_loss: 0.6031 - val_acc: 0.8080


Epoch 6/8
2s - loss: 0.4914 - acc: 0.8388 - val_loss: 0.5991 - val_acc: 0.8120


Epoch 7/8
2s - loss: 0.4583 - acc: 0.8538 - val_loss: 0.5837 - val_acc: 0.8260


Epoch 8/8
2s - loss: 0.4286 - acc: 0.8686 - val_loss: 0.5450 - val_acc: 0.8340



In [166]:
np.array(hist.history['val_acc'])

array([ 0.48 ,  0.689,  0.75 ,  0.717,  0.808,  0.812,  0.826,  0.834])

In [167]:
class Elemwise(Layer):
    def __init__(self, axis, is_mult, init='glorot_uniform', **kwargs):
        self.init = initializations.get(init)
        self.axis = axis
        self.is_mult = is_mult
        super(Elemwise, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dims = input_shape[1:]
        dims = [1] * len(input_dims)
        dims[self.axis] = input_dims[self.axis]
        self.b = self.add_weight(dims, self.init, '{}_bo'.format(self.name))
        self.built = True

    def call(self, x, mask=None):
        return x * self.b if self.is_mult else x + self.b

    def get_output_shape_for(self, input_shape):
        return input_shape

    def get_config(self):
        config = {'init': self.init.__name__, 'axis': self.axis}
        base_config = super(Dense, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))